In [108]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import math
import warnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing

warnings.filterwarnings("ignore")

In [63]:
# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Suppress specific ValueWarnings from statsmodels
warnings.filterwarnings("ignore", category=Warning, module="statsmodels")

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calculate_error_metrics(actual, forecasted):
    mae = np.mean(np.abs(actual - forecasted))
    mape = np.mean(np.abs((actual - forecasted) / actual)) * 100
    mse = np.mean((actual - forecasted) ** 2)
    rmse = np.sqrt(mse)
    return mae, mape, mse, rmse

cluster_model = {}
n_cluster = 8

In [91]:
# Getting Model for each cluster
for cluster in range(n_cluster):
    df = pd.read_csv(f'./cluster_{cluster}.csv', low_memory=False)
    
    df['launch_date'] = pd.to_datetime(df['launch_date'])
    df.sort_values('launch_date', ascending=True, inplace=True)
    df.set_index('launch_date', inplace=True)
    
    model = ExponentialSmoothing(df['target'], trend='add', seasonal='add', seasonal_periods=12)
    model_fit = model.fit()
    cluster_model[f'cluster-{cluster}'] = model_fit

In [99]:
# Process Sumbission
df_sub = pd.read_csv(
    './submission_data.csv',
    low_memory=False
)
df_clustering = pd.read_csv(
    './clustered_train_data.csv',
    low_memory=False
)

df_sub['ld'] = pd.to_datetime(df_sub['launch_date'], errors='coerce')
df_sub = df_sub[df_sub['ld'].dt.year == 2022]
df_sub['launch_date'] = pd.to_datetime(df_sub['launch_date'], errors='coerce')
df_clustering['launch_date'] = pd.to_datetime(df_clustering['launch_date'], errors='coerce')

In [100]:
drug_mapping = {}
# Iterate through each row in df_sub
for drug in df_sub['drug_id']:
    # Get all clusters that this cluster_nl appears in
    drugs = df_clustering[df_clustering['drug_id'] == drug]['cluster'].unique()
    # print(drugs)
    
    if len(drugs) > 1:
        drug_mapping[drug] = drugs.tolist()
    if len(drugs) == 0:
        print(f'non-clustered for drugs: {drugs}')

# Output the cluster_mapping dictionary
print("Duplicate Drug_Id In The Cluster \n", drug_mapping)

Duplicate Drug_Id In The Cluster 
 {'DRUG_ID_FADD': [1, 2, 4], 'DRUG_ID_E07F': [6, 7, 2, 3, 4], 'DRUG_ID_1A21': [2, 1, 4], 'DRUG_ID_1315': [1, 4, 2], 'DRUG_ID_2832': [1, 5, 4, 2], 'DRUG_ID_C25F': [6, 4], 'DRUG_ID_20C2': [3, 2, 1, 4], 'DRUG_ID_36C6': [0, 7], 'DRUG_ID_C6D4': [1, 4, 2, 3], 'DRUG_ID_4C1D': [2, 4, 1], 'DRUG_ID_0C4E': [1, 4, 3, 2], 'DRUG_ID_98D8': [1, 3, 2], 'DRUG_ID_473B': [1, 7, 6, 4], 'DRUG_ID_B4D3': [1, 5, 2, 4], 'DRUG_ID_EC3A': [2, 1], 'DRUG_ID_52A5': [7, 1, 2, 4], 'DRUG_ID_45DC': [1, 2, 4], 'DRUG_ID_D637': [6, 7, 1, 5, 3, 4], 'DRUG_ID_482D': [0, 7, 6], 'DRUG_ID_B15F': [6, 7, 1, 4], 'DRUG_ID_3F2A': [1, 2, 4], 'DRUG_ID_24A2': [6, 7, 3], 'DRUG_ID_C48E': [3, 1, 2, 4], 'DRUG_ID_B628': [1, 2], 'DRUG_ID_45AB': [5, 1, 2, 4], 'DRUG_ID_B050': [1, 2, 4], 'DRUG_ID_A874': [6, 5, 1, 7, 2], 'DRUG_ID_1C54': [1, 5, 6, 2], 'DRUG_ID_79D8': [1, 5, 2], 'DRUG_ID_AEE7': [1, 5, 2, 4], 'DRUG_ID_C6BA': [1, 2, 4], 'DRUG_ID_B0E9': [1, 2, 4], 'DRUG_ID_7DA2': [1, 5, 2], 'DRUG_ID_8AB0': [6, 7], 'DRU

In [101]:
cluster_nl_mapping = {}
# Iterate through each row in df_sub
for cluster_nl in df_sub['cluster_nl']:
    # Get all clusters that this cluster_nl appears in
    cluster_nls = df_clustering[df_clustering['cluster_nl'] == cluster_nl]['cluster'].unique()
    # print(cluster_nls)
    
    if len(cluster_nls) > 1:
        cluster_nl_mapping[cluster_nl] = cluster_nls.tolist()
    if len(cluster_nls) == 0:
        print(f'non-clustered for cluster_nl: {cluster_nl}')

# Output the cluster_mapping dictionary
print(cluster_nl_mapping)

{}


In [102]:
clustering_map = {}
n_cluster = 8
for cluster in range(n_cluster):
    data = pd.read_csv(
        f'./cluster_{cluster}.csv',
        low_memory=False
    )
    data['launch_date'] = pd.to_datetime(data['launch_date'], errors='coerce')
    latest = data.iloc[-1]
    clustering_map[f'cluster-{cluster}'] = {'list': data['launch_date'].to_list(), 'latest_date': latest['launch_date'], 'data': data}

In [120]:
# Generate Output By cluster_nl
for idx, data in df_sub.iterrows():
    cluster_nl = data['cluster_nl']
    cluster_nls = df_clustering[df_clustering['cluster_nl'] == cluster_nl]['cluster'].unique()
    selected_cluster = cluster_nls[0]
    launch_date = data['launch_date']
    latest_launch_date_cluster = clustering_map[f'cluster-{selected_cluster}']['latest_date']

    value = None
    
    if(launch_date > latest_launch_date_cluster):
        span_months = (launch_date.year - latest_launch_date_cluster.year) * 12 + (launch_date.month - latest_launch_date_cluster.month)
        model = cluster_model[f'cluster-{selected_cluster}']
        forecast = model.forecast(steps=span_months)
        value = forecast.values[0]
    else:
        cluster_data = clustering_map[f'cluster-{selected_cluster}']['data']
        closest_date = cluster_data[cluster_data['launch_date'] < launch_date]['launch_date'].max()

        retrain_data = cluster_data[cluster_data['launch_date'] <= closest_date]
        retrain_data['launch_date'] = pd.to_datetime(retrain_data['launch_date'])
        retrain_data.sort_values('launch_date', ascending=True, inplace=True)
        retrain_data.set_index('launch_date', inplace=True)
        retrain_model = ExponentialSmoothing(retrain_data['target'], trend='add', seasonal='add', seasonal_periods=12)
        retrain_model_fit = retrain_model.fit()

        span_months = (launch_date.year - closest_date.year) * 12 + (launch_date.month - closest_date.month)
        forecast = retrain_model_fit.forecast(steps=1)
        value = forecast.values[0]
        
    df_sub.at[idx, 'target'] = value
    print(f'Row {idx} - Forecast: {value}')
    
print("------------- Finish -------------")

Row 0 - Forecast: 1.0814362013540635
Row 1 - Forecast: 1.1006221790608628
Row 2 - Forecast: 1.060390610209867
Row 3 - Forecast: 1.0176151895692296
Row 4 - Forecast: 1.015242350326937
Row 5 - Forecast: 1.0826411672618
Row 6 - Forecast: 1.0176151895692296
Row 7 - Forecast: 1.0987940561068177
Row 8 - Forecast: 1.0164568552529138
Row 9 - Forecast: 1.0206831888646597
Row 10 - Forecast: 1.1321028077878783
Row 11 - Forecast: 1.1574453204094017
Row 12 - Forecast: 1.015343354087526
Row 13 - Forecast: 1.4312107570761567
Row 14 - Forecast: 1.0871118944623335
Row 15 - Forecast: 1.0448968168554646
Row 16 - Forecast: 1.1321028077878783
Row 17 - Forecast: 1.0862014514053282
Row 18 - Forecast: 1.20007875029646
Row 19 - Forecast: 1.0862014514053282
Row 21 - Forecast: 1.1321028077878783
Row 22 - Forecast: 1.1164227113609684
Row 23 - Forecast: 1.1321028077878783
Row 24 - Forecast: 1.0421008079327299
Row 25 - Forecast: 1.1105940053672756
Row 26 - Forecast: 1.0408381038793022
Row 27 - Forecast: 1.431210757

In [123]:
df_sub.drop(columns=['forecast'], inplace=True)  

In [124]:
df_sub.to_csv('result_2022.csv')

In [126]:
# Merged

submission_template = pd.read_csv('submission_template.csv')
result_2022 = pd.read_csv('result_2022.csv')

# Merge the DataFrames on 'date' and 'cluster_nl'
merged = submission_template.merge(
    result_2022[['date', 'cluster_nl', 'target']],  # Select only relevant columns
    on=['date', 'cluster_nl'],
    how='left'  # Use 'left' to keep all rows in submission_template
)

# Update the 'target' column in the submission template
submission_template['prediction'] = merged['target']

# Save the updated submission template
submission_template.to_csv('submission_2022.csv', index=False)

print("Updated submission_2022.csv has been saved.")

Updated submission_2022.csv has been saved.
